In [3]:
import nesmdb
import pickle
import os
import inspect
import tempfile
import subprocess
import numpy as np

In [4]:
def openScore (fileName, fileFolder = 'databases/nesmdb24_seprsco/train/'):
    with open(fileFolder+fileName, 'rb') as f:
        rate, nsamps, seprsco = pickle.load(f)
    return 24., nsamps+50000000, seprsco

In [5]:
def convertVGM(folderRead, folderWrite):
    directory = os.path.dirname(folderWrite)
    if not os.path.exists(directory):
        os.mkdir(directory) 
    if folderWrite is None:
        folderWrite=folderRead  
    i = 0
    for fileName in os.listdir(folderRead):
        seprsco = openScore(fileName, folderRead)
        np.savetxt("foo"+str(i)+".csv", seprsco[2], delimiter=",")
        exprsco = nesmdb.score.seprsco_to_exprsco(seprsco)
        rawsco = nesmdb.score.exprsco_to_rawsco(exprsco)
        ndf = nesmdb.score.rawsco_to_ndf(rawsco)
        ndr = nesmdb.vgm.ndf_to_ndr(ndf)
        vgm = nesmdb.vgm.ndr_to_vgm(ndr)
        
        with open(os.path.join(folderWrite, fileName.split(".")[0]+".vgm"), "w") as vf:
            vf.write(vgm)
        i +=1

In [6]:
convertVGM("pkl2saved2/", "savedVGM2/")

In [8]:
bin_dir = os.path.dirname(inspect.getfile(nesmdb))
bin_fp = os.path.join('.', bin_dir, 'vgm2wav.exe')

In [7]:
print(bin_fp)

.\nesmdb\vgm2wav\vgm2wav.exe


In [10]:
os.path.isfile(bin_fp)

True

In [7]:
vf = open("tmp.nsf", "w")
vf.write(vgm)

In [8]:
vf.close()